#### Python Version
This neural network runs on Python 3.12 to ensure compatability with its dependencies. If you are running this notebook in a virtual environment, ensure you have the correct runtime selected by running the below cell.

In [ ]:
!python --version

#### Install Dependencies
Installs the following dependencies for use in the notebook:
* **Torch:** The model is built using the PyTorch framework (this is also what limits the Python version to <= 3.12)
* **Torchvision:** Has functions for handling and preparing datasets for PyTorch models
* **Kaggle:** Download datasets from an online repository

In [ ]:
%pip install torch
%pip install torchvision
%pip install kaggle

#### Download and Prepare Datasets for Use
> Prior to running this code block, ensure you have a Kaggle API key stored locally in `~/.kaggle/kaggle.json`. Visit the Kaggle website for information on how to acquire an API key.
This neural network combines data from two datasets:
* The Breast Ultrasound Images (BUSI) Dataset (Al-Dhabyani W, Gomaa M, Khaled H, Fahmy A. Dataset of breast ultrasound images. Data in Brief. 2020 Feb;28:104863. DOI: 10.1016/j.dib.2019.104863.)
* Vuppala Adithya Sairam's Ultrasound Breat Images for Breast Cancer dataset, for which he has not provided a source other than the fact that it was aggregated from various open breast cancer ultrasound datasets

The BUSI dataset had an additional "normal" class of ultrasounds that had no tumors, but these are deleted as the purpose of this model is to identify whether a detected tumor is malignant of benign. Both datasets have "benign" and "malignant" images which are aggregated together. Sairam's dataset was already split into test and evaluation datasets, but these were combined as this notebook randomly splits the datasets later on.

In [ ]:
!kaggle datasets download "aryashah2k/breast-ultrasound-images-dataset" -p "./data"
!unzip data/breast-ultrasound-images-dataset.zip
!rm -rf Dataset_BUSI_with_GT/normal
!mv Dataset_BUSI_with_GT/* data
!rm -rf Dataset_BUSI_with_GT
!rm -rf data/breast-ultrasound-images-dataset.zip
!find data -type f -name "*_mask*.png" -delete

!kaggle datasets download "vuppalaadithyasairam/ultrasound-breast-images-for-breast-cancer" -p "./data"
!unzip data/ultrasound-breast-images-for-breast-cancer.zip
!mv "ultrasound breast classification/train/benign"/* data/benign
!mv "ultrasound breast classification/val/benign"/* data/benign
!mv "ultrasound breast classification/train/malignant"/* data/malignant
!mv "ultrasound breast classification/val/malignant"/* data/malignant
!rm -rf "ultrasound breast classification"
!rm -rf data/ultrasound-breast-images-for-breast-cancer.zip

#### Import Necessary Dependencies
The following dependencies are imported:
* `torch`: Contains various functions for identifying GPUs and manipulating Tensors
* `torch.nn`: Contains various prebuilt layers and classes to develop a deep learning network
* `torch.nn.functional`: Contains implementations of activation functions that add non-linearity to a deep learning network
* `torch.optim`: Contains optimizers used in model training
* `time`: Used in displaying metrics while training the model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

#### Select Device for Training
The following cell selects the best available device for training, testing, and performing inferences with the AI model. If a CUDA GPU is available, all calculations will be performed on the GPU. If an M-series Mac is used, PyTorch's MPS backend is used. Otherwise, all calculations will be done on the CPU.

In [ ]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

print(f"Device: {device}")

#### Load and Transform Datasets
The `torchvision` library is used to load and transform the data. The data is turned into a labeled dataset with the following labels:
* Images in `data/benign` will have a label `0`
* Images in `data/malignant` will have a label `1`

Images in the dataset are also transformed. They are converted to Grayscale (ultrasounds are in black and white anyway, so training on 3 color channels is a waste of computation power), transformed to Tensor shapes, and normalized to have a mean and standard deviation of 0.5. The data set is then randomly split into a train and test dataset, with the train dataset containing `80%` of the original dataset and the test dataset containing the remaining `20%`. These two datasets are then loaded, with the training dataset being randomly shuffled every epoch.

A batch size of `1` is used for both datasets as PyTorch expects batches to contain data of the same size. Since this model is designed to handle images of any size and the datasets are shuffled, a batch size of `1` is used to avoid any errors regarding data size mismatches within batches.

In [ ]:
from torchvision import datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader, random_split

transform = v2.Compose([
    v2.Grayscale(num_output_channels=1),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomRotation(30),
    v2.Resize((300, 300)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5], std=[0.5])
])

dataset = datasets.ImageFolder(root="data", transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

#### Define Neural Network Architecture
The model is a Convolutional Neural Network with Adaptive Average Pooling, meaning it can take in grayscale images of any size and identify a set number of features present within those images. These features are then passed through a fully connected neural network to determine whether an ultrasound contains a benign or malignant tumor.

The model's design choices are as follows:
* The convolutional portion of the model utilizes a pattern of two `3x3` `Conv2d` convolutional layers in a row followed by a `MaxPool2d` max pooling layer. The sequential `Conv2d` layers allow for powerful feature identification (when compared to a single convolutional layer, even one with a larger `5x5` kernel size) before being reduced to its more prominent characteristics through `MaxPool2d` layers.
* 6 `Conv2d` layers going from `16` to `32` to `64` output features create a balance between preventing overfitting and extracting sufficient features to accuractly determine a tumor's maliciousness.
* The `Conv2d` layers have a padding of `1` to retain the same output size as input size. This made the model more simple to develop, and doesn't consume crazy amounts of VRAM as these layers' are eventually adaptively pooled together through the `AdaptiveAvgPool2d` layer.
    * The `Conv2d` outputs' sizes are only modified by the interspersed `MaxPool2d` layers, which halves their heights and widths.
* The `AdaptiveAvgPool2d` layer bridges the convolutional and fully connected portions of the model. After inputs pass through the convolutional portion of the model and before they are pooled by the `AdaptiveAvgPool2d` layer, they have a shape of `64 x H/8 x W/8`, where `H` and `W` are the original heights and widths of the inputs, respectively. Adaptive average pooling reduces this to a `64 x 1 x 1` shape, with the resultant `64` features being a generalization of the convolutional layers' output.
    * In effect, this allows the model to take in input grayscale ultrasound images of any dimension, as they're eventually reduced to a `64 x 1 x 1` shape.
* The fully connected portion of the model is sparse and simple to prevent overfitting. It contains a single `Dropout` layer with a probability of `50%` between its two `Linear` layers to further improve the model's generalizability.
* The activation function utilized throughout this network is the `LeakyReLU` activation function. It was discovered during the model's development, when `ReLU` was the activation function being utilized, that there was a potential of dead neurons impacting model accuracy. Switching to `LeakyReLU` with a negative slope of `0.01` prevented dead neurons from occurring and lead to a `~2%` accuracy gain.
* The model outputs logits instead of probabilities through a `Sigmoid` activation layer, as the custom loss function defined for this model is more computationally efficient when model outputs are logits.
    * However, these logit outputs can be passed through a `Sigmoid` activation function in the code handling inferences for user-friendly output as obviously loss functions are not of concern there.

In [ ]:
class BreastCancerCNN(nn.Module):
    def __init__(self):
        super(BreastCancerCNN, self).__init__()

        # Convolutional layers
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Dropout2d(0.3),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Dropout2d(0.3),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Dropout2d(0.3),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.3)
        )
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        # Fully connected and dropout layers
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(0.01),
            nn.Linear(32, 1),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

#### Identifying Dataset Biases
In the source images, there are fewer malignant tumor images than benign tumor images. Since the training and test datasets are randomly split fractions of the total ultrasound dataset, it's safe to assume that in the training data there are more instances of benign tumors than malignant. This cell is meant to verify that assumption.

In [ ]:
benign_count = 0
malignant_count = 0
for _, label in train_loader:
    if label.item() == 0:
        benign_count += 1
    else:
        malignant_count += 1

print(f"Benign Image Count: {benign_count}")
print(f"Malignant Image Count: {malignant_count}")

#### Addressing Dataset Biases
Because there tend to be fewer examples of malignant tumors, it's harder for the neural network to identify malignant tumors compared to benign ones. Regular cross-entropy or binary cross-entropy loss functions don't address this issue, but their variant focal loss does. The focal loss formula looks like this:
$$
FL(p_t) = -\alpha_t(1-p_t)^\gamma\log(p_t)
$$
$
\newline p_t = \text{Probability of the input being of label } t
\newline \alpha_t = \text{Hyperparameter from } [0, 1] \text{ that scales down the loss of the label with fewer training instances. In binary classification tasks } \alpha_t = \alpha \text{ if } p_t = p \text{ and } \alpha_t = (1 - \alpha) \text{ if } p_t = (1 - p)
\newline \gamma = \text{Hyperparameter } \geq 0 \text{ that scales down the loss of easily identifiable labels to focus on training harder ones}
$

When adapted for binary classification tasks, the focal loss formula can look like the following:
$$
FL(p, y) = -\alpha y(1-p)^\gamma\log(p) - (1 - \alpha) (1-y)(p)^\gamma\log(1-p)
$$
$
\newline y = \text{Whether the label is 1 (malignant) or 0 (benign)}
$

In [ ]:
class BinaryFocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super(BinaryFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        # Convert to float for Binary Cross Entropy
        targets = targets.float()
        cross_entropy_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
        # Sigmoid the inputs to convert them into probabilities
        p = torch.sigmoid(inputs)
        # Since targets is either 0 or 1, this returns the probability for each possible outcome as either targets or (1 - targets) is 0 in these equations
        p_t = p * targets + (1 - p) * (1 - targets)
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        # Actually apply the focal loss formula
        focal_loss = alpha_t * (1 - p_t) ** self.gamma * cross_entropy_loss

        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss

#### Initialize Neural Network, Loss Function, Optimizer, and Epoch Counts
The loss function chosen is the aforementioned `BinaryFocalLoss` function. Since the majority label is the `0` label (images of benign tumors), an `alpha` of `0.25` allows its $\alpha_t = 0.75$ and thus its loss to be scaled by a factor of `0.75`, which is 3 times the scaling of the loss of the minority `1` label (images of malignant tumors), which has $\alpha_t = 1 - 0.75 = 0.25$. A `gamma` of 4 was selected to aggressively account for the imbalance in benign and malignant tumor images, as its more important for a cancer screening model to identify malignant tumors than benign ones.

The `AdamW` optimizer is selected as it adds effective handling of a small weight decay to prevent overfitting. A lower learning rate of `3e-4` was selected over the standard `1e-3` due to a need for increased accuracy.

`40` epochs were selected to result in an improved accuracy over the previous `20` training epochs used while still not taking exhorbinant amounts of time to train the model.

In [ ]:
cancer_net = BreastCancerCNN()
cancer_net = cancer_net.to(device)
loss_fn = BinaryFocalLoss(alpha=0.2, gamma=4)
optimizer = optim.AdamW(cancer_net.parameters(), lr=3e-4, weight_decay=1e-4)
num_epochs = 40

#### Train the Model
The below cell trains the model utilizing the loss function, optimizer, and number of epochs defined in the previous cell. The training loop also reports every `200` images the current epoch, the current image within the epoch, the total loss across the past `200` images, and the time elapsed since the model was trained on the last `200` images.

In [ ]:
for epoch in range(num_epochs):
    i = 0
    start_time = time.time()
    cancer_net.train()
    current_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cancer_net(inputs)
        loss = loss_fn(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()

        current_loss += loss.item()
        if i % 100 == 99 or i == len(train_loader) - 1:
            end_time = time.time()
            print(f"[Epoch: {epoch + 1}/{num_epochs}, Batch: {i + 1}/{len(train_loader)}] Loss: {current_loss:0.5f}, Time Elapsed: {end_time - start_time:0.5f}s")
            current_loss = 0.0
            start_time = end_time
        i += 1

print("Training complete!")

#### Evaluate the Model
The below cell block evaluates the model's **accuracy**, **precision**, **recall**, and **F1 score** with a strict threshold of `50%` confidence required to flag a tumor as malignant. Typical cancer applications would utilize a lower threshold to ensure that any false negatives don't slip through the cracks, but a stricter threshold is used here to overexaggerate how inaccurate the model is in order to further improve upon its development. Both the training and test datasets' performance is evaluated here, so as to check whether the model has overfitted or not.

For this model, **recall** is the most important metric as it measures the percentage of positives that were identified correctly. Higher **recall** means that fewer positive results slipped through the cracks. 

In [ ]:
threshold = 0.4

def evaluate_model(model, data_loader, threshold, data_name="dataset"):
    total = 0
    correct = 0
    total_positive = 0
    predicted_positive = 0
    predicted_positive_correct = 0
    with torch.no_grad():
        model.eval()
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = torch.sigmoid(outputs.data)
            predicted = (predicted > threshold).long()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_positive += (labels == 1).sum().item()
            predicted_positive += (predicted == 1).sum().item()
            predicted_positive_correct += (predicted == labels and predicted == 1).sum().item()
            
    accuracy = correct/total
    precision = predicted_positive_correct/predicted_positive
    recall = predicted_positive_correct/total_positive
    print(f"{data_name.title()} Accuracy: {correct}/{total} => {accuracy:0.7f}")
    print(f"{data_name.title()} Precision: {predicted_positive_correct}/{predicted_positive} => {precision:0.7f}")
    print(f"{data_name.title()} Recall: {predicted_positive_correct}/{total_positive} => {recall:0.7f}")
    print(f"{data_name.title()} F1 Score: {(2 * precision * recall/(precision + recall)):0.7f}")

evaluate_model(cancer_net, test_loader, threshold, "test data")

#### Load Cross-Validation Dataset

In [ ]:
!kaggle datasets download "sayedmeeralishah/breast-cancer-segmentation-dataset-preprocessed" -p "./cross_validation"
!unzip cross_validation/breast-cancer-segmentation-dataset-preprocessed
!mv "Breast-canser_preprocessed dataset/benign" cross_validation
!mv "Breast-canser_preprocessed dataset/malignant" cross_validation
!find cross_validation -type f -name "*_mask.png" -delete

!rm -rf "Breast-canser_preprocessed dataset"
!rm -rf cross_validation/breast-cancer-segmentation-dataset-preprocessed.zip

#### Evaluate Against Cross-Validation Data

In [ ]:
cross_validation_dataset = datasets.ImageFolder(root="cross_validation", transform=transform)
cross_validation_loader = DataLoader(cross_validation_dataset, batch_size=1, shuffle=False)
evaluate_model(cancer_net, cross_validation_loader, threshold, "cross validation")

#### Save Weights
Save the model's weights to a `.pth` file for later use, or in case the Jupyter Notebook kernel needs to be restarted to free up VRAM.

In [ ]:
torch.save(cancer_net.state_dict(), "breast_cancer_cnn.pth")

#### Perform an Inference
The following code cell loads an image (given a path to the image) into the model and performs an inference on it, returning a dictionary of the classification given to the image by the model and the model's confidence on that classification. The confidence is expressed as a decimal ranging from `0` to `1`. Multiplying it by `100` will result in a percentage.

In [ ]:
import types
from PIL import Image

def infer(self, img_path: str, threshold):
    self.eval()

    inference_transforms = v2.Compose([
        v2.Grayscale(num_output_channels=1),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.5], std=[0.5]),
    ])
    with Image.open(img_path) as pillow_image:
        # Unsqueeze adds an extra dimension to emulate a batch of size 1
        image = inference_transforms(pillow_image).unsqueeze(0).to(device)
        output = self(image)
        prediction = torch.sigmoid(output.data).item()
        if prediction > threshold:
            return {"label": "malignant", "confidence": prediction}
        else:
            return {"label": "benign", "confidence": 1 - prediction}

cancer_net.infer = types.MethodType(infer, cancer_net)


#### Example Inference
The below code cell loads the saved model weights and performs an inference based on an image path from the test data

In [ ]:
cancer_net.load_state_dict(torch.load("breast_cancer_cnn.pth"))
cancer_net.infer("data/benign/benign (1).png", 0.5)